In [12]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
import json
import requests
import pandas as pd
import csv
from pprint import pprint

In [2]:
# importing mongo and checking database names
mongo = MongoClient(port=27017)
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'establishments', 'fruits_db', 'fruitsdb', 'local', 'met', 'petsitly_marketing', 'player2_weather', 'travel_db', 'uk_food', 'weather_database']


In [3]:
# Define  filepath containing Stormglass API Key
file_path = './keys/stormglass.txt'

# Read the API key from the file
with open(file_path, 'r') as file:
    stormglass_api_key = file.read().strip()

In [54]:
# importing mongo, setting database to player2_weather
mongo = MongoClient('mongodb://localhost:27017')
db = mongo['player3_weather']
# creating or calling collection
dorianpost = db['dorianpost']
# url and parameters for stormglass
endpoint = 'https://api.stormglass.io/v2/weather/point'
params = {
            'lat': 34.2104,
            'lng': -77.8868,
            'params': 'airTemperature',
            'start': '2020-08-24',
            'end': '2020-09-10'
            
        }
headers = {
        'Authorization': stormglass_api_key
    }
# capturing response from stormglass
response = requests.get(endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()
# inserting it to designated collection in player_2 database
dorianpost.insert_one(data)
# receipt
if dorianpost.count_documents({}) > 0:
    print("Data inserted into dorianfirstyear collection.")
else:
    print("Failed to insert data.")


Data inserted into dorianfirstyear collection.


In [55]:
# checking out structure of data collected
data = dorianpost.find()
for document in data:
    pprint(document)

{'_id': ObjectId('65bc4ad30e43a888fbeb7217'),
 'hours': [{'airTemperature': {'noaa': 29.75, 'sg': 29.75},
            'time': '2020-08-24T00:00:00+00:00'},
           {'airTemperature': {'noaa': 29.68, 'sg': 29.68},
            'time': '2020-08-24T01:00:00+00:00'},
           {'airTemperature': {'noaa': 29.62, 'sg': 29.62},
            'time': '2020-08-24T02:00:00+00:00'},
           {'airTemperature': {'noaa': 29.55, 'sg': 29.55},
            'time': '2020-08-24T03:00:00+00:00'},
           {'airTemperature': {'noaa': 29.52, 'sg': 29.52},
            'time': '2020-08-24T04:00:00+00:00'},
           {'airTemperature': {'noaa': 29.48, 'sg': 29.48},
            'time': '2020-08-24T05:00:00+00:00'},
           {'airTemperature': {'noaa': 29.45, 'sg': 29.45},
            'time': '2020-08-24T06:00:00+00:00'},
           {'airTemperature': {'noaa': 29.48, 'sg': 29.48},
            'time': '2020-08-24T07:00:00+00:00'},
           {'airTemperature': {'noaa': 29.51, 'sg': 29.51},
            't

In [56]:
# naming collection as a list
data = list(dorianpost.find())
#dumping it to json file
json_data = json.dumps(data, default=str)
# selecting where json will be stored
with open('Resources/dorian_2020.json', 'w') as json_file:
    json_file.write(json_data)

In [57]:
# CREATING CSV
# AIR TEMP

# creating csv columns
data = dorianpost.find_one({}, {"_id": 0, "hours": 1})['hours']
# deciding location
csv_file = 'Resources/dorian_2020.csv'
# and writing first set of data to csv
fields = ['time', 'airTemperature_noaa', 'airTemperature_sg']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
    for entry in data:
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        writer.writerow(row_data)

In [233]:
# CREATING CSV
# WATER TEMP

# creating csv columns
data = dorian_willwat.find_one({}, {"_id": 0, "hours": 1})['hours']
# deciding location
csv_file = 'Resources/dorian_abaco_watertemp.csv'
# writing first of three captured data sets based on year of data occurrence
fields = ['time', 'waterTemperature_noaa', 'waterTemperature_sg']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
    for entry in data:
        row_data = [
            entry['time'],
            entry['waterTemperature']['noaa'],
            entry['waterTemperature']['sg']
        ]
        writer.writerow(row_data)

In [59]:
# APPENDING CSV
# AIR TEMP

# reading json
with open('Resources/dorian_2020.json') as file:
    data = json.load(file)

# labeling json subsequent data sets - second_dataset = 2019, third_dataset = 2020
second_dataset = data[1]
third_dataset = data[2]

# where the csv will be stored
csv_file = 'Resources/dorian_2020.csv'

# header fields
fields = ['time', 'airTemperature_noaa', 'airTemperature_sg']

# opening existing csv and using 'a' to append instead of 'w' for write
with open(csv_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header fields
    writer.writerow(fields)
    
    # in selected dataset, iterate over entries and write to csv
    for entry in third_dataset['hours']:
        # pulling each entry's data
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        writer.writerow(row_data)

In [235]:
# APPENDING CSV
# WATER TEMP

# reading json
with open('Resources/dorian_abaco_watertemp.json') as file:
    data = json.load(file)

# labeling json subsequent data sets - second_dataset = 2019, third_dataset = 2020
second_dataset = data[1]
third_dataset = data[2]

# where the csv will be stored
csv_file = 'Resources/dorian_abaco_watertemp.csv'

# header fields
fields = ['time', 'waterTemperature_noaa', 'waterTemperature_sg']

# opening existing csv and using 'a' to append instead of 'w' for write
with open(csv_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header fields
    writer.writerow(fields)
    
    # in selected dataset, iterate over entries and write to csv
    for entry in third_dataset['hours']:
        # pulling each entry's data
        row_data = [
            entry['time'],
            entry['waterTemperature']['noaa'],
            entry['waterTemperature']['sg']
        ]
        writer.writerow(row_data)